\_\_call\_\_

In [ ]:
from collections import defaultdict


class BetterCountMissing:
    def __init__(self):
        self.added = 0

    def __call__(self): # Entry point to the class's primary behavior
        self.added += 1
        return 0

increments = [
    ("red", 5),
    ("blue", 17),
    ("orange", 9),
]

counter = BetterCountMissing()
result = defaultdict(counter)  # Relies on __call__
for key, amount in increments:
    result[key] += amount
assert counter.added == 3
assert callable(counter)


Single dispatch as an alternative to OOP

In [4]:
# Functional-style rather than class centric
# "Single dispatch is an alternative approach for achieving dynamic 
# dispatch using functions instead of method polymorphism, making it 
# possible to bring related functionality closer together in source code."
import functools

@functools.singledispatch
def my_print(value):
    raise NotImplementedError

@my_print.register(int)
def _(value):
    print("Integer!", value)

@my_print.register(float)
def _(value):
    print("Float!", value)

@functools.singledispatch
def my_evaluate(node):
    raise NotImplementedError

class Integer:
    def __init__(self, value):
        self.value = value

@my_evaluate.register(Integer)
def _(node):
    return node.value

class Add:
    def __init__(self, left, right):
        self.left = left
        self.right = right

@my_evaluate.register(Add)
def _(node):
    left = my_evaluate(node.left)
    right = my_evaluate(node.right)
    return left + right

class Multiply:
    def __init__(self, left, right):
        self.left = left
        self.right = right

@my_evaluate.register(Multiply)
def _(node):
    left = my_evaluate(node.left)
    right = my_evaluate(node.right)
    return left * right

tree = Multiply(
    Add(Integer(3), Integer(5)),
    Add(Integer(4), Integer(7)),
)
result = my_evaluate(tree)
print(result)

88


Class method polymorphism to call a class method on different types of objects

In [7]:
import os
from threading import Thread


class Worker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self, other):
        raise NotImplementedError

class GenericInputData:
    def read(self):
        raise NotImplementedError

    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError
    
class PathInputData(GenericInputData):
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        with open(self.path) as f:
            return f.read()

    @classmethod
    def generate_inputs(cls, config):
        data_dir = config["data_dir"]
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))


class GenericWorker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self, other):
        raise NotImplementedError

    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers
    
class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count("\n")

    def reduce(self, other):
        self.result += other.result

def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

    first, *rest = workers
    for worker in rest:
        first.reduce(worker)
    return first.result

def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return execute(workers)

config = {"data_dir": 'classes_dummy_data'}
result = mapreduce(LineCountWorker, PathInputData, config)
print(f"There are {result} lines")

There are 18 lines


Private attributes

In [20]:
class MyObject:
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10

    def get_private_field(self):
        return self.__private_field
    
    def __private_method(self):
        print("Private Method Successfully Run")
    
foo = MyObject()
try:
    foo.__private_method()
except Exception as e:
    print("Error", e)

foo._MyObject__private_method()

Error 'MyObject' object has no attribute '__private_method'
Private Method Successfully Run


Descriptor Protocol

In [23]:
class Grade:
    def __set_name__(self, owner, name):
        print(f'__set_name__ called with owner: {owner} and name: {name}')
        # Determine the name for the internal attribute (e.g., _math_grade)
        self.internal_name = "_" + name

    def __get__(self, instance, instance_type):
        print(f'__get__ called with owner: {instance} and name: {instance_type}')
        if instance is None:
            return self
        # Retrieve the value from the instance dictionary
        return getattr(instance, self.internal_name)

    def __set__(self, instance, value):
        print(f'__set__ called with owner: {instance} and name: {value}')
        # Reusable validation logic
        if not (0 <= value <= 100):
            raise ValueError("Grade must be between 0 and 100")
        # Save the value to the instance using the internal name
        setattr(instance, self.internal_name, value)

class Exam:
    # Reusing the descriptor for multiple attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

__set_name__ called with owner: <class '__main__.Exam'> and name: math_grade
__set_name__ called with owner: <class '__main__.Exam'> and name: writing_grade
__set_name__ called with owner: <class '__main__.Exam'> and name: science_grade


In [24]:
first_exam = Exam()

In [25]:
first_exam.math_grade = 78

__set__ called with owner: <__main__.Exam object at 0x107b64ec0> and name: 78


In [26]:
first_exam.writing_grade = 89


__set__ called with owner: <__main__.Exam object at 0x107b64ec0> and name: 89


In [27]:
print(first_exam.__dict__)

{'_math_grade': 78, '_writing_grade': 89}
